## Using Langchain to create a custom prompt

We've taken our data and used knn search to find articles close to our topic. Now we can use that information to help provide insight to our LLM and use that to help provide context to our chat bot.

Let's run our search from the last notebook again. 

> **Reminder** For this data we used _OpenAI_ but our embeddings but this could we could have used any embedding algorithm. You can find other options on https://huggingface.co and similar websites.

Let's once again preform a similarity search on our data.

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from opensearchpy import OpenSearch

load_dotenv()

index_name = "openai_wikipedia_index"
connection_string = os.getenv("OPENSEARCH_SERVICE_URI")

# Create the client with SSL/TLS enabled, but hostname verification disabled.
opensearch_client = OpenSearch(connection_string, use_ssl=True, timeout=100)

# Define model
EMBEDDING_MODEL = "text-embedding-ada-002"

# Define the Client
openaiclient = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
    )

def search(
        embedding:list[float],
        index_name:str=index_name,
        num_results:int=5,
        k_distance:int=10
) -> str:
    """Preform a search using the embedding and return the top 5 results."""

    return opensearch_client.search(
        index = index_name,
        body = {
            "size": num_results,
            "query" : {
                "knn" : {
                    "content_vector":{
                        "vector": embedding,
                        "k": k_distance,
                    }
                }
            }
        }
    )



We can create embeddings for our question and preform a similarity search around pistacio being a good pizza topping.

In [ ]:
# Define questions
question = "Is pistachio ice cream good?"

question_embedding = openaiclient.embeddings.create(input=question + "pizza toppings", model=EMBEDDING_MODEL).data[0].embedding
docs = search(question_embedding)


Next we'll use the results from our search to augment our request. We'll also provide some guidance on how to present the information.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate


llm = ChatOpenAI()
# llm = ChatOllama()


top_hit_summary = [doc['_source']['text'] for doc in opensearch_response["hits"]["hits"]]

template="""
You are an assistant for a school that trains Pizza restaurant franchise owners and chefs.

Your goal is to helpo them find both traditional and unique ingredients for their pizza recipes.

Your suggestions should be based on the following criteria:
 - Ingredients should be able to be added to the pizza as a topping or sauce
 - Ingredients should be able to be consumed by humans
 - Flavor profiles should be compatible with pizza

Comment on their potential taste, texture, and how they might pair with other ingredients.

Respond to this Question: {question}

Don't suggest other ingredients.

Use ONLY the data below to influence your answer
{data}

Keep your answer to limited to no more than 2 sentences.
"""

prompt = PromptTemplate.from_template(template=template)

chain = (
    {"question": RunnablePassthrough(), "data": RunnablePassthrough()} | prompt | llm
)

response = chain.invoke({"question": question, "data": top_hit_summary})
print(response)

## Conclusion

OpenSearch is a powerful tool providing both text and vector search capabilities. Used alongside LangChain allows you to craft personalized AI applications able to augment the context based on semantic search. LangChain's extensive modularity allows you to choose your
